In [1]:
import os
import sys
from glob import glob

# compat
sys.path.append("../src/")

from geojson_processing import geojson_to_mask
import numpy as np
import time

annos_of_interest: list[str] = [
        "Tissue",
        "Tumor_vital",
        "diffuse_tumor_growth_in_soft_tissue",
        "Angioinvasion",
        "Tumor_necrosis",
        "Tumor_regression",
    ]
base_path = "/data2/projects/DigiStrudMed_sklein/DigiStrucMed_Braesen/all_data/RCC-TA-038.016flipped"


In [2]:
from cucim import CuImage

hxw = CuImage(base_path + ".svs")
# generate mask from geojson
gt_arr = geojson_to_mask(base_path+".json", annos_of_interest, hxw.shape[:2])["Tissue"]

DEBUG:rasterio.env:Entering env context: <rasterio.env.Env object at 0x7f694ae03d60>
DEBUG:rasterio.env:Starting outermost env
DEBUG:rasterio.env:No GDAL environment exists
DEBUG:rasterio.env:New GDAL environment <rasterio._env.GDALEnv object at 0x7f690e101870> created
DEBUG:rasterio._filepath:Installing FilePath filesystem handler plugin...
DEBUG:rasterio._env:GDAL_DATA found in environment.
DEBUG:rasterio._env:PROJ_DATA found in environment.
DEBUG:rasterio._env:Started GDALEnv: self=<rasterio._env.GDALEnv object at 0x7f690e101870>.
DEBUG:rasterio.env:Entered env context: <rasterio.env.Env object at 0x7f694ae03d60>
DEBUG:rasterio._io:Path: _UnparsedPath(path='MEM:::DATAPOINTER=140079921475600,PIXELS=90599,LINES=80376,BANDS=1,DATATYPE=Byte'), mode: r+, driver: None
DEBUG:rasterio._base:Nodata success: 0, Nodata value: 0.000000
DEBUG:rasterio.env:Exiting env context: <rasterio.env.Env object at 0x7f694ae03d60>
DEBUG:rasterio.env:Cleared existing <rasterio._env.GDALEnv object at 0x7f690e

In [8]:
import pyvips
start = time.time()
vips_img = pyvips.Image.tiffload(base_path+"-Tissue.tif", page=0)
vips_arr = vips_img.numpy()
print(time.time() - start)
print(np.equal(gt_arr, vips_arr).all())

start = time.time()
vips_img_at_1 = pyvips.Image.tiffload(base_path+"-Tissue.tif", page=1)
vips_arr = vips_img.numpy()
print(time.time() - start)


DEBUG:pyvips:Loaded binary module _libvips
DEBUG:pyvips:Module generated for libvips 8.13
DEBUG:pyvips:Linked to libvips 8.13
DEBUG:pyvips:Inited libvips
DEBUG:pyvips.voperation:VipsOperation.call: operation_name = tiffload
DEBUG:pyvips.voperation:VipsOperation.call: match_image = None
DEBUG:pyvips.vobject:VipsObject.set: name = filename, value = /data2/projects/DigiStrudMed_sklein/DigiStrucMed_Braesen/all_data/RCC-TA-038.016flipped-Tissue.tif
DEBUG:pyvips.vobject:VipsObject.set: name = page, value = 0
DEBUG:pyvips.vobject:VipsObject.get: name = out
DEBUG:pyvips.vobject:VipsObject.get: name = interpretation
DEBUG:pyvips.vobject:VipsObject.get: name = width
DEBUG:pyvips.vobject:VipsObject.get: name = height
DEBUG:pyvips.vobject:VipsObject.get: name = format
DEBUG:pyvips.vobject:VipsObject.get: name = bands
DEBUG:pyvips.vobject:VipsObject.get: name = interpretation
DEBUG:pyvips.voperation:VipsOperation.call: result = <pyvips.Image 90599x80376 uchar, 1 bands, b-w>


In [3]:
import tifffile
import zarr

start = time.time()
tf_img = tifffile.imread(base_path+".svs",  series=0, level=0) # series arg needed?
print(time.time() - start)

start = time.time()
tf_img_at_1 = tifffile.imread(base_path+".svs",  series=0, level=1) # series arg needed?
print(time.time() - start)




DEBUG:numcodecs:Registering codec 'zlib'
DEBUG:numcodecs:Registering codec 'gzip'
DEBUG:numcodecs:Registering codec 'bz2'
DEBUG:numcodecs:Registering codec 'lzma'
DEBUG:numcodecs:Registering codec 'blosc'
DEBUG:numcodecs:Registering codec 'zstd'
DEBUG:numcodecs:Registering codec 'lz4'
DEBUG:numcodecs:Registering codec 'astype'
DEBUG:numcodecs:Registering codec 'delta'
DEBUG:numcodecs:Registering codec 'quantize'
DEBUG:numcodecs:Registering codec 'fixedscaleoffset'
DEBUG:numcodecs:Registering codec 'packbits'
DEBUG:numcodecs:Registering codec 'categorize'
DEBUG:numcodecs:Registering codec 'pickle'
DEBUG:numcodecs:Registering codec 'base64'
DEBUG:numcodecs:Registering codec 'shuffle'
DEBUG:numcodecs:Registering codec 'bitround'
DEBUG:numcodecs:Registering codec 'msgpack2'
DEBUG:numcodecs:Registering codec 'crc32'
DEBUG:numcodecs:Registering codec 'adler32'
DEBUG:numcodecs:Registering codec 'json2'
DEBUG:numcodecs:Registering codec 'vlen-utf8'
DEBUG:numcodecs:Registering codec 'vlen-bytes

44.80091881752014
3.2311997413635254


In [32]:
store = tifffile.imread(base_path+".svs", aszarr=True) 
z = zarr.open(store, mode='r')
print(z[0])

location = (300, 400)
size = (512,512)


region: np.ndarray = np.asarray(z[0][location[0]:location[0] + size[0], location[1]:location[1] + size[1]]) # I think we need to remove the 2

store.close()

<zarr.core.Array '/0' (80376, 90599, 3) uint8 read-only>


In [33]:
type(store)

tifffile.tifffile.ZarrTiffStore

In [36]:
dir(store)

['_MutableMapping__marker',
 '__abstractmethods__',
 '__annotations__',
 '__class__',
 '__class_getitem__',
 '__contains__',
 '__del__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_chunkmode',
 '_chunks',
 '_data',
 '_dtype_str',
 '_empty_chunk',
 '_filecache',
 '_fillvalue',
 '_getitem',
 '_indices',
 '_is_writable',
 '_json',
 '_maxworkers',
 '_multiscales',
 '_ndindex',
 '_parse_key',
 '_setitem',
 '_squeeze',
 '_store',
 '_transform',
 '_value',
 '_writable',
 'clear',
 'close',
 'flush',
 'get',
 'is_multiscales',
 'items',
 'key

In [31]:
type(region)

numpy.ndarray

In [19]:
store.

<tifffile.ZarrTiffStore @0x00007F689DC0FFA0>

In [18]:
type(z)

zarr.hierarchy.Group

In [20]:
dir(z)

['_MutableMapping__marker',
 '__abstractmethods__',
 '__class__',
 '__class_getitem__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__setattr__',
 '__setitem__',
 '__setstate__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_array_iter',
 '_array_nosync',
 '_attrs',
 '_chunk_store',
 '_create_dataset_nosync',
 '_create_group_nosync',
 '_create_nosync',
 '_delitem_nosync',
 '_empty_like_nosync',
 '_empty_nosync',
 '_full_like_nosync',
 '_full_nosync',
 '_info',
 '_ipython_key_completions_',
 '_item_path',
 '_key_prefix',
 '_meta',
 '_meta_array',
 '_move_nosync

In [29]:
z.basename

''

In [27]:
z.meta_array

array(5.e-324)

In [27]:
ts_img.dimensions

(90599, 80376)

In [33]:
np.equal(np.asarray(hxw.read_region(level=0)), ts_arr).all()

True

In [31]:
ts_img: TiffSlide = TiffSlide(base_path+".svs") # series arg needed?
ts_arr = ts_img.read_region(level=0, location=(0,0), size= ts_img.dimensions,as_array=True)

In [30]:
from tiffslide import TiffSlide
start = time.time()
ts_img: TiffSlide = TiffSlide(base_path+"-Tissue.tif") # series arg needed?
ts_arr = ts_img.read_region(level=0, location=(0,0), size= ts_img.dimensions,as_array=True)
print(time.time() - start)
print(np.equal(np.expand_dims(gt_arr, axis=-1), ts_arr).all())

start: float = time.time()
ts_img_at_1: TiffSlide = TiffSlide(base_path+"-Tissue.tif") # series arg needed?
ts_arr: np.ndarray = ts_img_at_1.read_region(level=1, location=(0,0), size= ts_img.dimensions,as_array=True)
print(time.time() - start)


start = time.time()
region: np.ndarray  = ts_img.read_region((300, 400), 0, (512, 512), as_array=True)
print(time.time() - start)


7.816846609115601
True
3.394305467605591
0.0020143985748291016


In [ ]:
# NOTE: This is a bit confusing, check the other two first
from opentile import OpenTile
start = time.time()
ot_img = OpenTile.open(base_path+"-Tissue.tif",  (512,512)) # series arg needed?
ot_arr = ot_img.tile = ot_img.get_tile(0, (0, 0))
ot_img.close()
print(time.time() - start)

print(np.equal(gt_arr, ot_arr).all())

start = time.time()
ot_img = OpenTile.open(base_path+"-Tissue.tif",  (512,512)) # series arg needed?
ts_arr = ts_img_at_1.read_region((0,0), 0, (0,0), as_array=True)
print(time.time() - start)


start = time.time()
region = ts_img.read_region((300, 400), 0, (512, 512), as_array=True)
print(time.time() - start)


try with multiprocessing

In [2]:
import tifffile
import zarr
from tiffslide import TiffSlide

def r_tf(path: str)-> np.ndarray:

    tf_img = tifffile.imread(path,  series=0, level=0) # series arg needed?
    return tf_img

def tile_tf(path:str)-> np.ndarray:
    store = tifffile.imread(path, aszarr=True) 
    z = zarr.open(store, mode='r')

    location = (300, 400)
    size = (512,512)

    region= z[0][location[0]:location[0] + size[0], location[1]:location[1] + size[1]] # I think we need to remove the 2

    store.close()

    return region

def r_ts(path: str):

    ts_img: TiffSlide = TiffSlide(path) # series arg needed?
    ts_arr = ts_img.read_region(level=0, location=(0,0), size= ts_img.dimensions,as_array=True)

    return ts_arr

def tile_ts(path: str):

    ts_img: TiffSlide = TiffSlide(path) # series arg needed?
    ts_arr = ts_img.read_region(level=0, location=(300, 400), size= (512,512),as_array=True)

    return ts_arr

import multiprocessing as mp
from multiprocessing import Pool
from glob import glob
import os

with Pool(5) as p:
    start = time.time()
    print([i.shape for i in p.map(tile_tf,
        glob(os.path.join(os.path.dirname(base_path), "*.tif"))[:10]
    )])
    print(time.time() - start)
with Pool(5) as p:

    start = time.time()
    print([i.shape for i in p.map(r_tf,
        glob(os.path.join(os.path.dirname(base_path), "*.tif"))[:10]
    )])
    print(time.time() - start)

with Pool(5) as p:
    start = time.time()

    print([i.shape for i in p.map(tile_ts,
        glob(os.path.join(os.path.dirname(base_path), "*.tif"))[:10]
    )])
    print(time.time() - start)

with Pool(5) as p:
    start = time.time()
    print([i.shape for i in p.map(r_ts,
        glob(os.path.join(os.path.dirname(base_path), "*.tif"))[:10]
    )])
    print(time.time() - start)


DEBUG:numcodecs:Registering codec 'zlib'
DEBUG:numcodecs:Registering codec 'gzip'
DEBUG:numcodecs:Registering codec 'bz2'
DEBUG:numcodecs:Registering codec 'lzma'
DEBUG:numcodecs:Registering codec 'blosc'
DEBUG:numcodecs:Registering codec 'zstd'
DEBUG:numcodecs:Registering codec 'lz4'
DEBUG:numcodecs:Registering codec 'astype'
DEBUG:numcodecs:Registering codec 'delta'
DEBUG:numcodecs:Registering codec 'quantize'
DEBUG:numcodecs:Registering codec 'fixedscaleoffset'
DEBUG:numcodecs:Registering codec 'packbits'
DEBUG:numcodecs:Registering codec 'categorize'
DEBUG:numcodecs:Registering codec 'pickle'
DEBUG:numcodecs:Registering codec 'base64'
DEBUG:numcodecs:Registering codec 'shuffle'
DEBUG:numcodecs:Registering codec 'bitround'
DEBUG:numcodecs:Registering codec 'msgpack2'
DEBUG:numcodecs:Registering codec 'crc32'
DEBUG:numcodecs:Registering codec 'adler32'
DEBUG:numcodecs:Registering codec 'json2'
DEBUG:numcodecs:Registering codec 'vlen-utf8'
DEBUG:numcodecs:Registering codec 'vlen-bytes

[(512, 512), (512, 512), (512, 512), (512, 512), (512, 512), (512, 512), (512, 512), (512, 512), (512, 512), (512, 512)]
0.3256664276123047


In [2]:
import rasterio
from rasterio.windows import Window

with rasterio.open(base_path+'-Tissue.tif') as src:

    w = src.read(1, window=Window(0, 0, 512, 512))

DEBUG:rasterio.env:Entering env context: <rasterio.env.Env object at 0x7fc8fdd72ef0>
DEBUG:rasterio.env:Starting outermost env
DEBUG:rasterio.env:No GDAL environment exists
DEBUG:rasterio.env:New GDAL environment <rasterio._env.GDALEnv object at 0x7fc8fdd72680> created
DEBUG:rasterio._filepath:Installing FilePath filesystem handler plugin...
DEBUG:rasterio._env:GDAL_DATA found in environment.
DEBUG:rasterio._env:PROJ_DATA found in environment.
DEBUG:rasterio._env:Started GDALEnv: self=<rasterio._env.GDALEnv object at 0x7fc8fdd72680>.
DEBUG:rasterio.env:Entered env context: <rasterio.env.Env object at 0x7fc8fdd72ef0>
DEBUG:rasterio._base:Sharing flag: 0
/home/sklein/mambaforge/envs/sklein/lib/python3.10/site-packages/rasterio/__init__.py:304: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
DEBUG:rasterio._base:Nodata success: 0, Nodata value: 0.000000
DEBU

In [3]:
w

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)